In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import pandas as pd
import numpy as np
import os

# =========================================================
# 1. 설정 및 데이터 로드
# =========================================================
base_path = '/content/drive/MyDrive/Colab Notebooks/2025-2 데이터애널리틱스'
input_file = 'cohort_ver151_reorder_col.csv'           # 입력 파일
output_file = 'cohort_ver129_target_log_transformed.csv' # 결과 파일

path_in = os.path.join(base_path, input_file)
path_out = os.path.join(base_path, output_file)

print(f"📂 데이터 로드 중... ({input_file})")
if not os.path.exists(path_in):
    raise FileNotFoundError(f"❌ 파일을 찾을 수 없습니다: {path_in}")

df = pd.read_csv(path_in)

# [중요] 원본 컬럼 순서 저장 (순서 변경 방지)
original_columns = df.columns.tolist()

# =========================================================
# 2. 타겟 변수 로그 변환 (덮어쓰기)
# =========================================================
print("🔄 target_time_to_next 컬럼에 로그 변환(log1p) 적용 중...")

# 존재 여부 확인
if 'target_time_to_next' in df.columns:
    # np.log1p(x) = log(1 + x) 사용 (0인 경우 0으로 보존, 안전함)
    # 기존 컬럼에 값을 그대로 덮어씌움 -> 컬럼명 유지됨
    df['target_time_to_next'] = np.log1p(df['target_time_to_next'])
else:
    print("⚠️ 경고: target_time_to_next 컬럼이 없습니다.")

# =========================================================
# 3. 저장 (컬럼 순서 강제 유지)
# =========================================================
# 혹시 모를 순서 변경 방지를 위해 원본 순서대로 재정렬
df_final = df[original_columns].copy()

print("\n✅ 변환 결과 확인 (상위 5행)")
# 값이 로그 스케일(0~10 사이 소수)로 변했는지 확인
print(df_final[['hadm_id', 'target_time_to_next']].head().to_markdown(index=False))

df_final.to_csv(path_out, index=False)
print(f"\n🎉 작업 완료! 저장된 파일: {output_file}")
print("   - target_time_to_next 값이 로그 변환되었습니다.")
print("   - 컬럼명 및 순서는 원본과 동일합니다.")

📂 데이터 로드 중... (cohort_ver151_reorder_col.csv)
🔄 target_time_to_next 컬럼에 로그 변환(log1p) 적용 중...

✅ 변환 결과 확인 (상위 5행)
|     hadm_id |   target_time_to_next |
|------------:|----------------------:|
| 2.00063e+07 |              5.0814   |
| 2.00063e+07 |              0        |
| 2.00063e+07 |              0        |
| 2.00063e+07 |              0.693147 |
| 2.00063e+07 |              0        |

🎉 작업 완료! 저장된 파일: cohort_ver129_target_log_transformed.csv
   - target_time_to_next 값이 로그 변환되었습니다.
   - 컬럼명 및 순서는 원본과 동일합니다.
